In [ ]:
!nvidia-smi

Thu Apr 21 03:03:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 29.0 MB/s 
     |████████████████████████████████| 77 kB 8.1 MB/s 
     |████████████████████████████████| 596 kB 65.2 MB/s 
     |████████████████████████████████| 6.6 MB 55.2 MB/s 
     |████████████████████████████████| 895 kB 61.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Read the following link to get kaggle.json https://www.kaggle.com/general/74235


In [ ]:
%%shell
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 81% 33.0M/41.0M [00:00<00:00, 36.0MB/s]
100% 41.0M/41.0M [00:00<00:00, 45.9MB/s]


In [ ]:
!unzip /content/fake-and-real-news-dataset.zip

Archive:  /content/fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [ ]:
from tqdm.auto import tqdm
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import re
from os.path import join
import os
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification,AutoModelForSequenceClassification,create_optimizer,get_scheduler,get_polynomial_decay_schedule_with_warmup
from transformers import AutoConfig
from itertools import chain
from collections import Counter
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.utils.data import Dataset
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
fake_news_folder="fake_news"
fake_new_dataset="raw_dataset"
fake_new_models="models"
name="bert-base-uncased"
framework='pytorch'

In [ ]:
root_path="/content/drive/MyDrive"
fake_news_folder_path=join(root_path,fake_news_folder)
fake_news_dataset_path=join(root_path,fake_news_folder,fake_new_dataset)
fake_news_models_path=join(root_path,fake_news_folder,fake_new_models)
model_path=f'{fake_news_models_path}/{name}/'
framework_path=f'{model_path}{framework}/'

In [ ]:
os.makedirs(model_path, exist_ok=True)
os.makedirs(fake_news_dataset_path, exist_ok=True)
os.makedirs(framework_path, exist_ok=True)

# Load fake news dataset from kaggle

In [ ]:
true=pd.read_csv("/content/True.csv")
false=pd.read_csv("/content/Fake.csv")

In [ ]:
true

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [ ]:
false

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [ ]:
true.groupby(['subject']).count()

,title,text,date
subject,,,
politicsNews,11272,11272,11272
worldnews,10145,10145,10145


In [ ]:
false.groupby(['subject']).count()

,title,text,date
subject,,,
Government News,1570,1570,1570
Middle-east,778,778,778
News,9050,9050,9050
US_News,783,783,783
left-news,4459,4459,4459
politics,6841,6841,6841


add label

In [ ]:
true['label']=[1]*len(true)
false['label']=[0]*len(false)

subjects of the both are not the same -> remove subject column

assume data column is not meaning

In [ ]:
df=pd.concat([true,false],axis=0)
df=df.drop(['subject','date'],axis=1)
df=df.sample(frac=1)

In [ ]:
df

,title,text,label
17427,"Canadian says child killed, U.S. wife raped du...",OTTAWA/TORONTO (Reuters) - A U.S.-Canadian cou...,1
15283,Graft suspected in Indonesian anti-corruption ...,JAKARTA (Reuters) - An Indonesian prosecutor h...,1
7640,Holocaust Survivor Says Donald Trump Reminds ...,Donald Trump is literally scaring Holocaust su...,0
13999,WATCH: HOME IMPROVEMENT’S TIM ALLEN Reminds Us...,NOTE TO TARGET There Aren t Any Feminine Produ...,0
19693,"China, Singapore look to put difficulties behi...",BEIJING (Reuters) - China and Singapore on Tue...,1
...,...,...,...
17337,UNREAL! OBAMA ADMIN PRESSURES LENDERS TO LOAN ...,We ve been doing articles on this for the past...,0
19625,FLASHBACK 2015: ANTI-GUN OBSESSED White House ...,Nothing says Thanksgiving or family like a...,0
16184,"In Kenya, polarizing election re-run rekindles...","MUHORONI, Kenya (Reuters) - Kenya s repeat pre...",1
3721,It’s Already Happening: Potential Trump Pick ...,Kansas Secretary of State Kris Kobach has repo...,0


In [ ]:
def merge(title,text):
  merged=[]
  for i in zip(title,text):
    if i[0].lower() not in i[1].lower():
      merged.append(i[0]+" "+i[1])
    else:
      merged.append(i[1])
  return merged

In [ ]:
merged=merge(df['title'].tolist(),df['text'].tolist())

In [ ]:
df['merged']=merged

In [ ]:
df

,title,text,label,merged
17427,"Canadian says child killed, U.S. wife raped du...",OTTAWA/TORONTO (Reuters) - A U.S.-Canadian cou...,1,"Canadian says child killed, U.S. wife raped du..."
15283,Graft suspected in Indonesian anti-corruption ...,JAKARTA (Reuters) - An Indonesian prosecutor h...,1,Graft suspected in Indonesian anti-corruption ...
7640,Holocaust Survivor Says Donald Trump Reminds ...,Donald Trump is literally scaring Holocaust su...,0,Holocaust Survivor Says Donald Trump Reminds ...
13999,WATCH: HOME IMPROVEMENT’S TIM ALLEN Reminds Us...,NOTE TO TARGET There Aren t Any Feminine Produ...,0,WATCH: HOME IMPROVEMENT’S TIM ALLEN Reminds Us...
19693,"China, Singapore look to put difficulties behi...",BEIJING (Reuters) - China and Singapore on Tue...,1,"China, Singapore look to put difficulties behi..."
...,...,...,...,...
17337,UNREAL! OBAMA ADMIN PRESSURES LENDERS TO LOAN ...,We ve been doing articles on this for the past...,0,UNREAL! OBAMA ADMIN PRESSURES LENDERS TO LOAN ...
19625,FLASHBACK 2015: ANTI-GUN OBSESSED White House ...,Nothing says Thanksgiving or family like a...,0,FLASHBACK 2015: ANTI-GUN OBSESSED White House ...
16184,"In Kenya, polarizing election re-run rekindles...","MUHORONI, Kenya (Reuters) - Kenya s repeat pre...",1,"In Kenya, polarizing election re-run rekindles..."
3721,It’s Already Happening: Potential Trump Pick ...,Kansas Secretary of State Kris Kobach has repo...,0,It’s Already Happening: Potential Trump Pick ...


Split raw dataset to training set, validating set, testing set

In [ ]:
X,test=train_test_split(df,test_size=0.2,stratify=df['label'])

In [ ]:
train,valid=train_test_split(X,train_size=0.87,stratify=X['label'])

In [ ]:
len(x_train)/len(x),len(x_valid)/len(x),len(x_test)/len(x)

(0.695977549111319, 0.10401354180587108, 0.20000890908280994)

Check data whether it is balanced or not

In [ ]:
for i in range(2):
  samples=train[train['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 16343 samples
label: 1	has 14905 samples


In [ ]:
for i in range(2):
  samples=valid[valid['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 2442 samples
label: 1	has 2228 samples


In [ ]:
for i in range(2):
  samples=test[test['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 4696 samples
label: 1	has 4284 samples


the number of Fake samples is greater than the last one

In [ ]:
train.to_csv(join(fake_news_dataset_path,"train.csv"),index=False)

In [ ]:
valid.to_csv(join(fake_news_dataset_path,"valid.csv"),index=False)

In [ ]:
test.to_csv(join(fake_news_dataset_path,"test.csv"),index=False)

# Clean and preprocessing

##Basic clean data

In [ ]:
train=pd.read_csv(join(fake_news_dataset_path,"train.csv"),index_col=False)
train=train.dropna(subset=['label', 'merged'])

valid=pd.read_csv(join(fake_news_dataset_path,"valid.csv"),index_col=False)
valid=valid.dropna(subset=['label', 'merged'])


In [ ]:
train

,title,text,label,merged
0,French envoy says Paris climate deal doesn't i...,PARIS (Reuters) - The Paris climate change dea...,1,French envoy says Paris climate deal doesn't i...
1,Bernie Sanders Has PERFECT Response To Man Ra...,Bernie Sanders smacked down a man at his Harle...,0,Bernie Sanders Has PERFECT Response To Man Ra...
2,"Concerned about refugees, U.N. experts add to ...",(Reuters) - U.N. human rights experts warned t...,1,"Concerned about refugees, U.N. experts add to ..."
3,Trump On The Verge Of Meltdown As FBI Clears ...,The FBI had just announced that it was clearin...,0,Trump On The Verge Of Meltdown As FBI Clears ...
4,Turkey determined to maintain EU accession pro...,ANKARA (Reuters) - Turkey is determined to pre...,1,Turkey determined to maintain EU accession pro...
...,...,...,...,...
31243,"Russian communists drop veteran, select surpri...",MOSCOW (Reuters) - The Russian Communist Party...,1,"Russian communists drop veteran, select surpri..."
31244,RUDY GIULIANI JUST BLEW Hillary’s Phony “Khant...,"To me, this is the logo of the Obama administ...",0,RUDY GIULIANI JUST BLEW Hillary’s Phony “Khant...
31245,Watch Bill Maher Deliver The Hard Truth So Ma...,"Win or lose, the fact that Donald Trump was ab...",0,Watch Bill Maher Deliver The Hard Truth So Ma...
31246,TUCKER CARLSON DESTROYS Smug Elector Who Refus...,https://youtu.be/7oOhwHG2Gb4,0,TUCKER CARLSON DESTROYS Smug Elector Who Refus...


In [ ]:
train.iloc[31246]['merged']

'TUCKER CARLSON DESTROYS Smug Elector Who Refuses To Vote For Trump [VIDEO] https://youtu.be/7oOhwHG2Gb4'

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [ ]:
list_stopwords=stopwords.words('english')

In [ ]:
def white_space_fix(text):
  return ' '.join(text.split())

In [ ]:
def normalize(s):
  def lower(text):
    return text.lower()
  
  def norm_apostrophe(text):
    return re.sub("’(\s|)","'",text)

  def replace_contraction(text):
    for k,v in contraction_mapping.items():
      text=text.replace(k,v)
    return text
  def remove_punc(text):
    return re.sub("\W"," ",text)

  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)

  def norm_number(text):
    new_text=text
    pattern = re.compile(r'\b((\d+)[.,\s]+(\d+))\b',re.UNICODE)

    founds=re.findall(pattern,new_text)
    for i in founds:
      new_text=re.sub(i[0],"".join(i[1:]),new_text)
    return new_text

  def remove_stopwords(text):
    new_text=text
    pattern=r'\b({})\b'.format(f'({"|".join(list_stopwords)})')
    regex = re.compile(pattern, re.UNICODE)
    new_text=re.sub(regex, ' ', new_text)
    return new_text

  def remove_nickname_twitter(text):
    pattern=r'\(@\b(.+)\b\)'
    return re.sub(pattern," ",text)

  def remove_link(text):
    pattern=r'\b((?:https?://)?(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)\b'
    return re.sub(pattern," ",text)

  def remove_single_character(text):
    return re.sub(r'\b([a-zA-Z])\b'," ",text)

  new_s=remove_link(s)
  new_s=remove_nickname_twitter(new_s)
  new_s=lower(new_s)
  new_s=norm_apostrophe(new_s)
  new_s=replace_contraction(new_s)
  # new_s=norm_number(new_s)
  new_s=remove_punc(new_s)
  new_s=remove_articles(new_s)
  new_s=remove_single_character(new_s)
  new_s=remove_stopwords(new_s)

  return white_space_fix(new_s).strip()


In [ ]:
train['merged']=train['merged'].map(normalize)

In [ ]:
def find_top_common_tokens(list_texts,top_n=30):
  tokens=list(chain.from_iterable([i.split() for i in list_texts]))
  counter=Counter(tokens)
  top_common_tokens=dict(counter.most_common(top_n))
  return list(top_common_tokens.keys())

In [ ]:
top_common_tokens=find_top_common_tokens(train['merged'].tolist())

In [ ]:
top_common_tokens

['trump',
 'said',
 'president',
 'would',
 'people',
 'one',
 'state',
 'new',
 'obama',
 'also',
 'reuters',
 'house',
 'clinton',
 'government',
 'donald',
 'states',
 'republican',
 'year',
 'white',
 'could',
 'united',
 'told',
 'campaign',
 'two',
 'election',
 'like',
 'party',
 'last',
 'news',
 'time']

In [ ]:
def remove_top_common_tokens(text):

  pattern=r'\b({})\b'.format(f'({"|".join(top_common_tokens)})')
  regex = re.compile(pattern, re.UNICODE)
  text=re.sub(regex, ' ', text)
  return white_space_fix(text).strip()

In [ ]:
train['merged']=train['merged'].map(remove_top_common_tokens)

In [ ]:
valid['merged']=valid['merged'].map(normalize)

In [ ]:
test['merged']=test['merged'].map(normalize)

In [ ]:
train=train[train['merged']!=""]
valid=valid[valid['merged']!=""]

In [ ]:
test=test[test['merged']!=""]

In [ ]:
len(train),len(valid),len(test)

(31244, 4669, 8976)

In [ ]:
train.to_csv(join(model_path,"train.csv"),index=False)
valid.to_csv(join(model_path,"valid.csv"),index=False)
test.to_csv(join(model_path,"test.csv"),index=False)

##Preprocessing

In [ ]:
train=pd.read_csv(join(model_path,"train.csv"),index_col=False)
valid=pd.read_csv(join(model_path,"valid.csv"),index_col=False)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
max_length=512

In [ ]:
train_batch_size=8
num_training_samples=len(train)
num_training_batches=int((num_training_samples/train_batch_size)+0.5)

In [ ]:
valid_batch_size=8
num_valid_samples=len(valid)
num_valid_batches=int((num_valid_samples/valid_batch_size)+0.5)

## Pytorch

In [ ]:
class FakeNewsDataset(Dataset):
    def __init__(self, input_ids, attention_mask,token_type_ids,labels,device):
        self.labels = labels
        self.input_ids=input_ids
        self.attention_mask=attention_mask
        self.device=device
        
    def __len__(self):
            return len(self.labels)
    def __getitem__(self, idx):
        label = self.labels[idx]
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        sample = {"input_ids": input_ids.to(device),'attention_mask':attention_mask.to(device), "labels": label.to(device)}
        return sample

In [ ]:
return_tensors='pt'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Train set

In [ ]:
x_train=tokenizer(train['merged'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [ ]:
y_train=torch.Tensor(train['label']).to(torch.long)

In [ ]:
train_dataset={i:x_train[i] for i in x_train}
train_dataset.update({"labels":y_train})

In [ ]:
train_dataset=FakeNewsDataset(**train_dataset,device=device)

In [ ]:
train_dataset=DataLoader(train_dataset,batch_size=train_batch_size,drop_last=True)

Valid set

In [ ]:
x_valid=tokenizer(valid['merged'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)#.to(device)

In [ ]:
y_valid=torch.Tensor(valid['label']).to(torch.long)#.to(device)

In [ ]:
valid_dataset={i:x_valid[i] for i in x_valid}
valid_dataset.update({"labels":y_valid})

In [ ]:
valid_dataset=FakeNewsDataset(**valid_dataset,device=device)

In [ ]:
valid_dataset=DataLoader(valid_dataset,batch_size=valid_batch_size,drop_last=True)

## Tensorflow

Train set

In [ ]:
x_train=tokenizer(train['merged'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [ ]:
y_train=tf.convert_to_tensor(train['label'],dtype=tf.int32)

In [ ]:
train_dataset={i:x_train[i] for i in x_train}
train_dataset.update({"labels":y_train})

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices(train_dataset)

In [ ]:
train_dataset=train_dataset.batch(train_batch_size, drop_remainder=True)

Valid set

In [ ]:
x_valid=tokenizer(valid['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [ ]:
y_valid=tf.convert_to_tensor(valid['label'],dtype=tf.int32)

In [ ]:
valid_dataset={i:x_valid[i] for i in x_valid}
valid_dataset.update({"labels":y_valid})
valid_dataset=tf.data.Dataset.from_tensor_slices(valid_dataset)

In [ ]:
valid_dataset=valid_dataset.batch(valid_batch_size, drop_remainder=True)

# Train

In [ ]:
epochs=5
num_training_steps=epochs * len(train_dataset)
num_warmup_steps=800
init_lr=1e-5
min_lr_ratio=1e-2

In [ ]:
num_training_steps,num_warmup_steps

(19525, 800)

## Pytorch

Load the pretrained model

In [ ]:
num_labels=len(train['label'].unique())
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels
model = AutoModelForSequenceClassification.from_pretrained(name,config=config).to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

- optimizer -> adam
- lr_schdeluer -> polynomial

In [ ]:
# checkpoint = torch.load("/content/drive/MyDrive/fake_news/models/bert-base-uncased/pytorch/ckpt4.pt")
# model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
optimizer=torch.optim.Adam(model.parameters(),lr=init_lr)
lr_scheduler=get_polynomial_decay_schedule_with_warmup(optimizer,num_warmup_steps,num_training_steps,lr_end =init_lr*min_lr_ratio)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
def save_model(epoch,model,optimizer,loss,path):
  torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss,
          }, join(path,f'ckpt{epoch}.pt'))

In [ ]:
def run_train(optimizer,dataloader,lr_scheduler):
  model.train(True)
  loop = tqdm(dataloader)
  
  loss=0
  for batch in dataloader:
    # model.zero_grad(set_to_none=True)
    optimizer.zero_grad()
    outputs=model(**batch)

    outputs.loss.backward()
    _loss=outputs.loss

    loss+=_loss.item()
    optimizer.step()
    lr_scheduler.step()
    loop.set_postfix({f'train_loss_per_batch':_loss.item()})
    loop.update()
    
  return loss/len(dataloader)

In [ ]:
def run_valid(dataloader):
  loop = tqdm(dataloader)
  model.eval()
  loss=0
  with torch.no_grad():
    for batch in dataloader:
      outputs=model(**batch)
      _loss=outputs.loss

      loss+=_loss.item()
      loop.set_postfix({f'valid_loss_per_batch':_loss.item()})
      loop.update()
  return loss/len(dataloader)

In [ ]:
for epoch in range(epochs):
  train_loss=run_train(optimizer,train_dataset,lr_scheduler)
  valid_loss=run_valid(valid_dataset)
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss}')
  save_model(epoch,model,optimizer,valid_loss,framework_path)

  0%|          | 0/3905 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

epoch: 0 train_loss: 0.08563762601981403 valid_loss: 0.015309230833614683


  0%|          | 0/3905 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

epoch: 1 train_loss: 0.0076412885601420035 valid_loss: 0.014835425387246963


  0%|          | 0/3905 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

## Tensorflow

In [ ]:
num_labels=len(train['label'].unique())
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels
model = TFAutoModelForSequenceClassification.from_pretrained(name,config=config)

In [ ]:
loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction=tf.keras.losses.Reduction.AUTO)

- optimizer -> adam
- lr_schdeluer -> polynomial

In [ ]:
optimizer,_=create_optimizer(init_lr=init_lr,num_train_steps =num_training_steps ,num_warmup_steps=num_warmup_steps,adam_epsilon =1e-8,min_lr_ratio =min_lr_ratio )

Define train_step function for training

In [ ]:
train_step_signature= [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, ), dtype=tf.int32),
]


@tf.function(input_signature=train_step_signature)
def train_step(input_ids, attention_mask,labels):
  

  with tf.GradientTape() as tape:

    outputs= model(input_ids=input_ids,attention_mask=attention_mask,training = True)
    loss=loss_fn(labels,outputs.logits)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

Defind test_step function for validation

In [ ]:
ckpt_path = framework_path
max_to_keep=5

ckpt = tf.train.Checkpoint(model=model,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=max_to_keep)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print(f'Loading the latest checkpoint from {ckpt_manager.latest_checkpoint}')
else:
  print('Initializing the new checkpoint')

In [ ]:
def test_step(input_ids, attention_mask,labels):
  
  outputs= model(input_ids=input_ids,attention_mask=attention_mask,training = False)
  loss=loss_fn(labels,outputs.logits)

  return loss

In [ ]:
def run_step(dataset,session='train'):
  
  step=train_step if session=='train' else test_step

  loop = tqdm(dataset)
  loss=0

  num_batches=0
  for batch in dataset:

    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask']
    labels=batch['labels']

    _loss=step(input_ids=input_ids,attention_mask=attention_mask,labels=labels)


    _loss=_loss.numpy()
    loss+=_loss
    num_batches+=1
    loop.set_postfix({f'{session}_loss_per_batch':_loss})
    loop.update()


  loss=loss/num_batches
  return loss

In [ ]:
min_loss=np.inf
for epoch in range(epochs):

  """TRAINING"""
  train_loss=run_step(train_dataset)

  """Validation"""
  valid_loss=run_step(valid_dataset,session='valid')
  
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss}')
  
  ckpt_save_path = ckpt_manager.save()
  print(f'Saving checkpoint for epoch {epoch} at {ckpt_save_path}')


#Load test set

In [ ]:
test=pd.read_csv(join(model_path,"test.csv"),index_col=False)
test

,title,text,label,merged
0,2017 NDAA: Obama Signs “Countering Disinformat...,21st Century Wire says In typical NDAA signing...,0,2017 ndaa obama signs countering disinformatio...
1,'Death to blasphemers' increasing as political...,"SWABI, Pakistan (Reuters) - Three police offic...",1,death blasphemers increasing political rallyin...
2,WATCH: IRAN Made VIDEOTAPE Of Secret $400 Mill...,It s starting to get old but Trump was right a...,0,watch iran made videotape secret 400 million r...
3,Searchers for Argentine submarine defy gale fo...,BUENOS AIRES (Reuters) - Searchers for an Arge...,1,searchers argentine submarine defy gale force ...
4,WATCH: Anderson Cooper HUMILIATES Trump Suppo...,Anderson Cooper only needed a single sentence ...,0,watch anderson cooper humiliates trump support...
...,...,...,...,...
8971,"EP #13: Patrick Henningsen LIVE – ‘Fake News, ...",In response to the establishment media s con...,0,ep 13 patrick henningsen live fake news fake c...
8972,It’s Not Over: Trump’s Tantrums Drive Senate ...,Trumpcare failed in the most hilarious and dra...,0,trump tantrums drive senate gop take aim yet h...
8973,Blackstone's Schwarzman sees historic regulato...,(Reuters) - Blackstone Group LP (BX.N) Chairma...,1,blackstone schwarzman sees historic regulatory...
8974,BREAKING: CNN PRODUCER Caught On Undercover Vi...,Project Veritas newest video from the America...,0,breaking cnn producer caught undercover video ...


In [ ]:
samples=test['merged'].tolist()

In [ ]:
num_labels=len(test['label'].unique())
num_labels

2

In [ ]:
labels=torch.Tensor(test['label'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(name)

Load model

# Pytorch

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained(name,config=config).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model.num_labels

2

In [ ]:
return_tensors='pt'

Load the weights

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/fake_news/models/bert-base-uncased/pytorch/ckpt1.pt")
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
max_length=512

In [ ]:
model.eval()
y_pred=[]
with torch.no_grad():
  loop = tqdm(range(len(samples)))
  for idx,sample in enumerate(samples):
    inputs=tokenizer(sample,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors).to(device)
    outputs=model(**inputs)
    prob=torch.nn.functional.softmax(outputs.logits,dim=1)
    preds=torch.argmax(prob,dim=1)
    y_pred.append(preds)
    loop.update()


  0%|          | 0/8976 [00:00<?, ?it/s]

In [ ]:
y_pred=torch.Tensor(y_pred)

In [ ]:
y_pred

tensor([0., 1., 0.,  ..., 1., 0., 0.])

In [ ]:
labels

tensor([0., 1., 0.,  ..., 1., 0., 0.], device='cuda:0')

In [ ]:
print(classification_report(labels,y_pred,digits=5))

              precision    recall  f1-score   support

         0.0    0.99936   0.99254   0.99594      4692
         1.0    0.99189   0.99930   0.99558      4284

    accuracy                        0.99577      8976
   macro avg    0.99562   0.99592   0.99576      8976
weighted avg    0.99579   0.99577   0.99577      8976



In [ ]:
confusion_matrix(labels,y_pred)

array([[4657,   35],
       [   3, 4281]])

In [ ]:
torch.where(y_pred!=labels)

(tensor([ 590,  896, 1495, 1577, 1741, 1748, 1907, 2414, 2608, 2637, 2858, 2965,
         3105, 3499, 3649, 3890, 4383, 4889, 5420, 5753, 5778, 5912, 5949, 5993,
         6475, 6476, 6874, 7305, 7331, 7554, 7580, 7976, 8064, 8433, 8624, 8708,
         8795, 8870]),)

the score is high but it is unrealistic -> the dataset is ideal

Try dataset from https://www.kaggle.com/competitions/fake-news/data

the f1 score is 63% ->much noise in dataset